## 如何使用tf.train.Saver() 把多个模型融合起来

**20171026更新：更正第2部分 -> 载入多个模型**
- 前版本错误说明：之前失误，写这个例子的时候，两个模型放在一块训练，即 model1， model2 的saver都会把另外一个模型的所有参数都进行保存。这样在后面重新导入模型的时候没有问题。但是实际上两个模型肯定是分开训练的，如果还是按照原来的写法，会发现在导入模型的时候，由于每个 ckpt 并没有保存另外一个模型的变量而报错。
- 现版本更正方法：重新定义两个 saver， 在定义的时候分别传入各模型的变量。

----

下面代码展示了怎么把多个独立训练好的模型联合起来fine-tune。因为比赛中没注意，导致最后有个模型没有保存好。下面的例子中用了两个模型，多个模型同样可以按照下面的方式进行。有几点需要注意：

- 在训练单个模型的时候，**一定一定一定要把模型的定义放在一个大的 variable_scope 中，这样在多个模型融合的时候才不会出现命名冲突。**
- 模型融合需要用到的变量，比如最后的一个全连接层，一定要写好接口。
- 将模型融合以后，一定一定一定要重新定义一个Saver()，这样才会把融合的大模型中所有的变量存到checkpoint中。

下面3部分分别表示：
- 1.分别训练单个模型（注意在不同的variable_scope中定义），保存单个模型。
- 2.载入多个模型，进行 fine-tune。融合以后定义新的 Saver()， 保存 fine-tune 好的模型。
- 3.载入 fine-tune 好的模型。

### 1.训练单个模型

In [1]:
# ** 1.分别训练单个模型（注意在不同的variable_scope中定义），保存单个模型。

import tensorflow as tf
import numpy as np
import os

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

import wd_1_1_cnn_concat.network as network1 

setting1 = network1.Settings()
W_embedding = np.random.randn(5, 6)


# 定义模型
with tf.variable_scope('model1') as vs:
    model1 = network1.TextCNN(settings=setting1, W_embedding=W_embedding)

sess.run(tf.global_variables_initializer())
# 保存第1个模型
ckpt_path1 = '../ckpt/saver_test/model1/model.ckpt'
if not os.path.exists(os.path.dirname(ckpt_path1)):
    os.makedirs(os.path.dirname(ckpt_path1))
save_path1 = model1.saver.save(sess, ckpt_path1, global_step=1)
print(save_path1)

../ckpt/saver_test/model1/model.ckpt-1


### restart the kernel， and train the second model

In [1]:
# ** 1.分别训练单个模型（注意在不同的variable_scope中定义），保存单个模型。

import tensorflow as tf
import numpy as np

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

import wd_3_bigru.network as network2

setting2 = network2.Settings()
W_embedding = np.random.randn(5, 6)


# 定义模型
with tf.variable_scope('model2') as vs:
    model2 = network2.BiGRU(settings=setting2, W_embedding=W_embedding)

sess.run(tf.global_variables_initializer())
    
# 保存第2个模型
ckpt_path2 = '../ckpt/saver_test/model2/model.ckpt'
save_path2 = model2.saver.save(sess, ckpt_path2, global_step=1)
print(save_path2)

../ckpt/saver_test/model2/model.ckpt-1


### 2.载入多个模型
**注意，先 restart kernel， 再执行下面的代码**

In [1]:
# ** 2.载入多个模型，进行 fine-tune。融合以后定义新的 Saver()， 保存 fine-tune 好的模型。

import tensorflow as tf
import numpy as np

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

import wd_1_1_cnn_concat.network as network1 
import wd_3_bigru.network as network2

setting1 = network1.Settings()
setting2 = network2.Settings()
W_embedding = np.random.randn(5, 6)


# 下面两个模型
with tf.variable_scope('model1') as vs:
    model1 = network1.TextCNN(settings=setting1, W_embedding=W_embedding)
    model1_vars = [v for v in tf.global_variables() if v.name.startswith(vs.name + '/')]

with tf.variable_scope('model2') as vs:
    model2 = network2.BiGRU(settings=setting2, W_embedding=W_embedding)
    model2_vars = [v for v in tf.global_variables() if v.name.startswith(vs.name + '/')]
    
# **导入第1个模型
# 创建 saver1，传入 model1_vars
saver1 = tf.train.Saver(model1_vars)
ckpt_path1 = '../ckpt/saver_test/model1/model.ckpt'
saver1.restore(sess, ckpt_path1 + '-' + str(1))
print(model1.y_pred)
    
# **导入第2个模型
saver2 = tf.train.Saver(model2_vars)
ckpt_path2 = '../ckpt/saver_test/model2/model.ckpt'
saver2.restore(sess, ckpt_path2 + '-' + str(1))
print(model2.y_pred)

# 创建新的变量
with tf.variable_scope('new_vars') as vs:
    y_pred = tf.concat([model1.y_pred, model2.y_pred], axis=1)
    new_vars = [v for v in tf.global_variables() if v.name.startswith(vs.name+'/')]
# 初始化新的变量
sess.run(tf.variables_initializer(new_vars))
print(y_pred)
# 创建新的 saver 保存大模型的所有参数
saver = tf.train.Saver(max_to_keep=1)
ckpt_path_all =  '../ckpt/saver_test/model_all/model.ckpt'
save_path_all = saver.save(sess, ckpt_path_all, global_step=1)
print(save_path_all)

INFO:tensorflow:Restoring parameters from ../ckpt/saver_test/model1/model.ckpt-1
Tensor("model1/out_layer/y_pred:0", shape=(?, 1999), dtype=float32)
INFO:tensorflow:Restoring parameters from ../ckpt/saver_test/model2/model.ckpt-1
Tensor("model2/out_layer/y_pred:0", shape=(?, 1999), dtype=float32)
Tensor("new_vars/concat:0", shape=(?, 3998), dtype=float32)
../ckpt/saver_test/model_all/model.ckpt-1


### 3.载入 fine-tune 好的大模型
**注意，先 restart kernel， 再执行下面的代码**

In [1]:
# ** 3.载入 fine-tune 好的模型。

import tensorflow as tf
import numpy as np

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

import wd_1_1_cnn_concat.network as network1 
import wd_3_bigru.network as network2

setting1 = network1.Settings()
setting2 = network2.Settings()
W_embedding = np.random.randn(5, 6)


# 下面两个模型
with tf.variable_scope('model1') as vs:
    model1 = network1.TextCNN(settings=setting1, W_embedding=W_embedding)

with tf.variable_scope('model2') as vs:
    model2 = network2.BiGRU(settings=setting2, W_embedding=W_embedding)

# 创建新的变量
with tf.variable_scope('new_vars') as vs:
    y_pred = tf.concat([model1.y_pred, model2.y_pred], axis=1)    

saver = tf.train.Saver(max_to_keep=1)
ckpt_path_all =  '../ckpt/saver_test/model_all/model.ckpt'
saver.restore(sess, ckpt_path_all + '-' + str(1))
print(y_pred)

INFO:tensorflow:Restoring parameters from ../ckpt/saver_test/model_all/model.ckpt-1
Tensor("new_vars/concat:0", shape=(?, 3998), dtype=float32)
